# WandB run data

In [8]:
import wandb
import os
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
fontsize = 12
plt.rc('font', size=fontsize)
plt.rc('axes', titlesize=fontsize)
plt.rcParams.update({'font.size': fontsize})

In [9]:
data_dir = "data"
run_prefix = 'wullli/flatland/'

In [10]:
run_map = {#"Ape-X TreeObs and Local Conflict": ["28maqwn2", "2g8ev19o", "32v0xigi"],
           "PPO TreeObs and Action Masking": ["1fp8rsnd", "2cw55h5o", "w9tyfmsv"],
           "Centralized Critic Transformer PPO TreeObs": ["14bkmmc1", "3ohw0yo6"],
           #"Centralized Critic PPO TreeObs": ["1wjmjc4v", "2ul4dpfr"],
           "SAC TreeObs": ["2dj4pn4r", "3u3h718q", "3r7t8jn7"]}

In [11]:
grouped_runs = {gn: [run_prefix + rid for rid in ids] for gn, ids in run_map.items()}

In [12]:
api = wandb.

AttributeError: module 'wandb' has no attribute 'Api'

In [ ]:
grouped_dfs = {}

for gn, run_paths in grouped_runs.items():
    dfs = []
    for i, run in enumerate(run_paths):
        run = api.run(run)
        data = run.history()
        dfs.append(data)
    grouped_dfs[gn] = dfs

In [ ]:
def plot_with_error_band(x_key: str, y_key: str, 
                         dfs: list, 
                         color='#CC4F1B', 
                         label=None):
    lengths = [len(df.values) for df in dfs]
    min_len = min(lengths)
    xs = np.stack([df[x_key].values[:min_len] for df in dfs])
    ys = np.stack([df[y_key].values[:min_len] for df in dfs])
    
    min_y = np.min(ys, axis=0)
    max_y = np.max(ys, axis=0)
    mean_y = np.mean(ys, axis=0)
    mean_x = np.mean(xs, axis=0)

    plt.plot(mean_x, mean_y, 'k', color=color, label=label)
    plt.fill_between(mean_x, min_y, max_y,
        alpha=0.15, edgecolor=color, facecolor=color)

In [ ]:
x_key = "timesteps_total"
y_key = "custom_metrics/percentage_complete_mean"

color_idx = 0
figure(num=None, figsize=(10, 5), dpi=300, facecolor='w', edgecolor='k')

plt.xlim((0, 1.5e7))
plt.ylim((0, 1))

for gn, dfs in grouped_dfs.items():
    color = list(mcolors.TABLEAU_COLORS.values())[color_idx]
    plot_with_error_band(x_key, y_key, dfs, color, label=gn)
    color_idx += 1

plt.yticks(ticks=np.arange(0, 1, 0.1))
plt.grid(axis='y', alpha=0.5)
plt.legend(loc="lower right")
plt.ylabel("Percentage complete")
plt.xlabel("Timesteps")
plt.title("")
plt.show()